# <오늘 할 것: 실습 테스트 1, 2, 3>

# 1. django와 low level db
- 1.1 카테고리, 페이지, 유저명으로 글 검색 예제
    - 1.1.1 왜 SQL문을 f rotation 처리하면 에러가 발생할까?
- 1.2 얼굴 인식 예제
    - 1.2.1 DB에서 유저명과 일치하는 이미지들 불러와서 띄우기
    - 1.2.2 이미지를 업로드하면 유저명의 디렉토리에 저장하기
    - 1.2.3 시간 남으면 AJAX로 구현한다
    - 1.2.4 시간 더 남으면 view class로도 해본다

# 1. django와 low level db

`settings.py`에 있는 db에 연결해주는 라이브러리가 있다.


In [1]:
from django.db import connection
from django.contrib.auth.models import User
from myboard import models 

In [2]:
cursor = connection.cursor()

cursor.execute('SELECT id, title, cnt FROM myboard_board')  #*을 쓰는 건 별로 좋지 않다(서버 부담)

results = cursor.fetchall()

print(type(results))

results[0]

<class 'list'>


(5, 'url 어떻게 해~~~', 3)

## SQLite에서 SQL 잠깐 연습하기

특정 유저가 작성한 글에 접근하고 싶다면?

`myboard_board` 테이블에는 유저명(`username`)은 없고 유저아이디(`id`)만 있기 때문에 그 둘이 매칭되어 있는 `auth_user` 테이블에서 유저명에 해당하는 유저아이디를 알아낸 후에 유저아이디로 `myboard_board`에서 검색할 수 있다.

즉, 유저명이 주어지면,

1. `auth_user`에서 유저명에 해당하는 유저아이디 가져오기
2. `myboard_board`에서 1에서 얻은 유저아이디에 해당하는 데이터 읽어오기

In [ ]:
SELECT id, title, cnt
FROM myboard_board
WHERE author_id=(
	SELECT id FROM auth_user WHERE username='hong')   /*조건절 안에는 유일값이 나와야한다!!!!*/

In [ ]:
SELECT b.id, title, cnt
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id AND username='hong'

같은 표현인데, 아래가 더 직관적이다.

## 주의할 것

`fetchone`이나 `fetchall` 메소드에서 `fetch` 개념은 **소모** 개념으로 작동하기 때문에 일단 한번 보여주고 나면 다음으로 넘어가버린다. 그래서 `fetchone`을 한번 한 뒤에 `fetchall`을 하면, 첫 데이터는 제외한 나머지를 보여주고, `fetchall`을 한번 하고 난 후에 다시 한번 `fetchall`을 하면 None이 리턴된다.

In [5]:
cursor.execute("""SELECT b.id, title, cnt, username
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id AND username='hong'""")

In [6]:
r1 = cursor.fetchone()

print(type(r1))
print(r1)

<class 'tuple'>
(5, 'url 어떻게 해~~~', 3, 'hong')


In [8]:
r_all = cursor.fetchall()

print(type(r_all))
print(r_all)

<class 'list'>
[(6, '기타 글 1', 8, 'hong'), (7, '기타 글 2', 4, 'hong'), (8, '이미지 업로드', 9, 'hong'), (9, '사진 올리기', 14, 'hong'), (13, 'test1~~~', 0, 'hong'), (14, 'test2~~~~', 0, 'hong'), (15, 'test3~~~~~~', 0, 'hong'), (16, 'test4~~~~', 0, 'hong'), (17, 'test5~~~~', 0, 'hong'), (18, 'test6', 0, 'hong'), (19, 'test7~~~~~', 0, 'hong')]


읽어온 데이터에는 field명이 포함되어 있지 않다는 문제가 있다.

`zip` 함수를 활용해서 간단하게 `field명 : field값` 쌍으로 이루어진 딕셔너리(들의 리스트)로 데이터들을 바꿔줄거다.

In [9]:
cursor.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

나중에 함수화할 때는 여기서 field명을 가져올 거다.

In [10]:
data = dict(zip(['id', 'title', 'cnt', 'username'], r_all[0]))

In [11]:
data

{'id': 6, 'title': '기타 글 1', 'cnt': 8, 'username': 'hong'}

In [20]:
#fetchall로 불러와서 field명 붙여서 딕셔너리화하는 함수
def dictfetchall(cursor):   #cursor는 execute 후, fecth 전의 cursor다
    desc = cursor.description
    return [
        dict(zip([element[0] for element in desc], row)) 
        for row in cursor.fetchall()
    ]

In [40]:
cursor.execute("""SELECT b.id, title, cnt, username, category
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id AND username= 'hong'""")

dictfetchall(cursor)

[{'id': 5,
  'title': 'url 어떻게 해~~~',
  'cnt': 3,
  'username': 'hong',
  'category': 'data'},
 {'id': 6, 'title': '기타 글 1', 'cnt': 8, 'username': 'hong', 'category': 'etc'},
 {'id': 7, 'title': '기타 글 2', 'cnt': 4, 'username': 'hong', 'category': 'etc'},
 {'id': 8,
  'title': '이미지 업로드',
  'cnt': 9,
  'username': 'hong',
  'category': 'data'},
 {'id': 9,
  'title': '사진 올리기',
  'cnt': 14,
  'username': 'hong',
  'category': 'data'},
 {'id': 13,
  'title': 'test1~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 14,
  'title': 'test2~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 15,
  'title': 'test3~~~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 16,
  'title': 'test4~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 17,
  'title': 'test5~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 18,
  'title': 'test6',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'}

## 1.1 카테고리, 페이지, 유저명으로 글 검색 예제

서버에 올려서 테스트하면 오래걸리니까 최대한 여기(django shell-plus)에서 테스트하고 django로 옮긴다.

In [29]:
category = "common"
page = 2
username = "hong"

In [32]:
cursor = connection.cursor()

In [41]:
SQL = """SELECT b.id, title, cnt, username, category
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id
AND username= 'hong'
AND category= 'common'
"""

cursor.execute(SQL)

dictfetchall(cursor)

[{'id': 13,
  'title': 'test1~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 14,
  'title': 'test2~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 15,
  'title': 'test3~~~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 16,
  'title': 'test4~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 17,
  'title': 'test5~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 18,
  'title': 'test6',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'},
 {'id': 19,
  'title': 'test7~~~~~',
  'cnt': 0,
  'username': 'hong',
  'category': 'common'}]

### 1.1.1 왜 SQL문을 f rotation으로 처리하면 문제가 발생할까?

In [44]:
SQL = f"""SELECT b.id, title, cnt, username, category
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id
AND username= {username}
AND category= {category}
"""

이렇게 쓰면 커서 객체에서 SQL을 `execute`했을 때 에러가 발생한다.

In [45]:
cursor.execute(SQL)

OperationalError: no such column: hong

그 이유는 값들이 **문자열이 아니라 변수명**으로 들어갔기 때문이다. f rotation으로 생성된 문자열을 출력해봤으면 바로 발견할 수 있었던 문제인데 바보 같았다.

In [47]:
print(SQL)

SELECT b.id, title, cnt, username, category
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id
AND username= hong
AND category= common



마지막 두줄에서 `'hong'`과 `'common'`으로 들어갔어야했다.

아래와 같이 작은 따옴표로 감싸주기만 하면 해결된다.

In [48]:
SQL = f"""SELECT b.id, title, cnt, username, category
FROM myboard_board b, auth_user u

WHERE b.author_id = u.id
AND username= '{username}'
AND category= '{category}'
"""

In [49]:
cursor.execute(SQL)

잘 된다. 분하다.....

## 1.2 얼굴 인식 예제

얼굴 사진을 업로드하면 db의 파일과 대조해서 로그인을 수행하는 기능을 추가할 거다. 업로드된 이미지 파일들은 `static` 폴더에서 따로 관리하고, 나머지 정보는 SQLite로 만든 테이블에서 관리한다.

SQLite에서는 3개 필드로 구성된 테이블을 만든다.
- 고유 id
- 유저명
- 이미지 파일 경로

In [ ]:
CREATE TABLE myboard_image(
id INTEGER PRIMARY KEY AUTOINCREMENT,
author_id INTEGER NOT NULL,
filename VARCHAR(50) NOT NULL,
FOREIGN KEY (author_id) REFERENCES auth_user(id)  /* 여타 테이블과 join 하기 위해*/
)

이제 SQL로 db에 입력하는 문장을 짜야한다. python에서는 model 객체를 생성해서 `create` 또는 11111을 활용하면 쉽게 생성할 수 있었는데, SQL은 좀 더 실험을 해봐야 알 수 있겠다.

마찬가지로 SQLite를 활용하면 SQL문을 어떻게 적성해야할지 힌트를 얻을 수 있다.

`데이터보기` 탭에서 `새 레코드` 버튼을 누르면 내가 입력한 내용에 맞추어 SQL문 예시를 작성해준다.

<img src=sql_ex.jpg?>

### 1.2.1 DB에서 유저명과 일치하는 이미지들 불러와서 띄우기

In [50]:
username = 'hong'

sql = f"""SELECT id from auth_user where username='{username}'"""

cursor.execute(sql)

author_id = cursor.fetchone()[0]

print(author_id)

3


In [51]:
filename = 'face02.jpg'

In [52]:
SQL = f"""
INSERT INTO "main"."myboard_image"
("author_id", "filename")
VALUES ({author_id}, '{filename}');
"""

In [54]:
cursor.execute(SQL)

In [60]:
SQL = f"""SELECT filename
    FROM myboard_image
    WHERE author_id = (SELECT id FROM auth_user WHERE username = '{username}')
    """
print(SQL)

SELECT filename
    FROM myboard_image
    WHERE author_id = (SELECT id FROM auth_user WHERE username = 'hong')
    


In [61]:
cursor = connection.cursor()

cursor.execute(SQL)
data = dictfetchall(cursor)
context = {'datas': data, 'username': username}

print(context)

{'datas': [{'filename': 'f1.jpg'}, {'filename': 'f2.jpg'}, {'filename': 'f3.jpg'}, {'filename': 'f4.jpg'}, {'filename': 'f5.jpg'}], 'username': 'hong'}


In [75]:
for d in data:
    print(d['filename'])

f1.jpg
f2.jpg
f3.jpg
f4.jpg
f5.jpg


In [69]:
from django.shortcuts import render
from django.test import RequestFactory
rf = RequestFactory()
request = rf.get('/myboard/photolist/')

In [70]:
response = render(request, "myboard/photolist.html", context)

print(response.content.decode())


hong  <br>

<form action="upload" method="post" enctype="multipart/form-data">
    <input type='hidden' name='csrfmiddlewaretoken' value='GiPcBAWodEOx7jSdvY7dy24Q2Bh50nayZ4f63MtrX0fXGqgIlyfybhJy68PNJzVz' />
    <input type="file" name="filename" /> <br>
    <input type="submit" value="사진전송"/> <br>
    <input type="hidden" name='username' value="hong"/>
</form>


<img src="/static/faces/hong/f1.jpg" width="100"> <br>

<img src="/static/faces/hong/f2.jpg" width="100"> <br>

<img src="/static/faces/hong/f3.jpg" width="100"> <br>

<img src="/static/faces/hong/f4.jpg" width="100"> <br>

<img src="/static/faces/hong/f5.jpg" width="100"> <br>



### 1.2.2 이미지를 업로드하면 유저명의 디렉토리에 저장하기

django에서 제공하는 `SimpleUploadedFile` 모듈을 활용하면 file 전송도 `RequestFactory`에서 실험해볼 수 있다.


### `SimpleUploadedFile`
- `SimpleUploadedFile('저장할 파일명', 바이너리로 읽은 이미지, content_type='image/jpeg')`

In [80]:
from django.core.files.uploadedfile import SimpleUploadedFile
f = open('static/faces/hong/f1.jpg', 'rb')

file_upload = SimpleUploadedFile('test1.jpg', f.read(), content_type = 'image/jpeg')

datas = {
    "filename": file_upload, "username": "hong"
}

request = rf.post('myboard/upload', data = datas, format='multipart')

In [81]:
file = request.FILES['filename']
filename = file._name

In [82]:
filename

'test1.jpg'

request 메세지가 제대로 생성되었다.

저장도 실험해본다.

In [85]:
fp = open(settings.BASE_DIR + f"/static/faces/{username}/" + filename, "wb")
for chunk in file.chunks():
    fp.write(chunk)
fp.close()

잘된다.

### 1.2.3 시간 남으면 AJAX로 구현한다

`사진 전송` 버튼을 눌러서 업로드가 실행될 때 새로 업로드된 이미지가 페이지 리프레시 없이 화면에 추가되도록 한다.

### 1.2.4 시간 더 남으면 view class로도 해본다